In [7]:

import xarray as xr
import rasterio
import numpy as np

In [8]:
def npy_to_map(input_arr, output_tif, proj_file,
               fill_value=np.nan, shift=0,
               dtype=rasterio.float32):
    """
    将一维 .npy 数组铺回到栅格地图中。
    - input_arr: path to .npy (1D array of length = number of valid pixels in proj_file)
    - output_tif: 输出 GeoTIFF 路径
    - proj_file: 用于投影和形状参照的已有 GeoTIFF
    - fill_value: 初始填充值（默认 np.nan）
    - shift: 在写入前对数据统一加的偏移量
    - dtype: 输出栅格的数据类型
    """
    # if not input_arr.lower().endswith(".npy"):
    #     return

    # 1) 读取参考栅格
    with rasterio.open(proj_file) as src:
        mask2D = src.read(1) >= 0
        transform = src.transform
        crs = src.crs
        profile = src.profile.copy()
        shape = src.shape

    # 2) 加载一维数组
    nonzeroes = np.where(mask2D)
    lumap = input_arr
    if lumap.ndim != 1:
        raise ValueError(f"{input_arr} 中的数组不是一维的")
    if len(lumap) != len(nonzeroes[0]):
        print(f"Warning: {input_arr} 的长度为 {len(lumap)}, proj_file 中有效像元数量为 {len(nonzeroes[0])}.")
        raise ValueError("lumap 的长度与 proj_file 中的有效像元数量不一致")

    # 3) 构建全图，并赋值
    themap = np.full(shape, fill_value=fill_value, dtype=float)
    themap[nonzeroes] = lumap + shift

    # 4) 把 +/- inf 都变成 np.nan
    themap[~np.isfinite(themap)] = np.nan

    # 5) 更新 profile 并写出
    profile.update({
        'dtype': dtype,
        'count': 1,
        'compress': 'lzw',
        'nodata': fill_value
    })
    with rasterio.open(output_tif, 'w', **profile) as dst:
        dst.write(themap.astype(dtype), 1)

    return output_tif

In [2]:
proj_file = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250831_Price_Task_NCI\Run_002_GHG_high_PERCENT_20_BIO_high_CUT_20\output\2025_09_01__21_48_14_RF5_2010-2050\out_2013\ammap_2013.tiff"


In [14]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high\2050\xr_total_cost_carbon_high_amortised_2050.nc"
input_xr = xr.open_dataarray(input_path)
input_arr = input_xr.to_numpy()
output_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\total_cost.tif"
npy_to_map(input_arr, output_tif, proj_file)

'N:\\LUF-Modelling\\LUTO2_XH\\LUTO2\\output\\20250908_Paper2_Results_NCI\\carbon_price\\4_tif\\total_cost.tif'

In [13]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high\2050\xr_total_carbon_high_2050.nc"
input_xr = xr.open_dataarray(input_path)
input_arr = input_xr.to_numpy()
output_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\carbon.tif"
npy_to_map(input_arr, output_tif, proj_file)

'N:\\LUF-Modelling\\LUTO2_XH\\LUTO2\\output\\20250908_Paper2_Results_NCI\\carbon_price\\4_tif\\carbon.tif'

In [29]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high\2050\xr_cost_ag_carbon_high_2050.nc"
input_xr = xr.open_dataarray(input_path)
input_arr = input_xr.sum(dim=['lm','lu']).to_numpy()
output_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\ag_cost.tif"
npy_to_map(input_arr, output_tif, proj_file)

'N:\\LUF-Modelling\\LUTO2_XH\\LUTO2\\output\\20250908_Paper2_Results_NCI\\carbon_price\\4_tif\\ag_cost.tif'

In [30]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high\2050\xr_cost_non_ag_carbon_high_2050.nc"
input_xr = xr.open_dataarray(input_path)
input_arr = input_xr.sum(dim=[d for d in input_xr.dims if d != "cell"]).to_numpy()
output_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\non_ag.tif"
npy_to_map(input_arr, output_tif, proj_file)

CPLE_AppDefinedError: Deleting N:/LUF-Modelling/LUTO2_XH/LUTO2/output/20250908_Paper2_Results_NCI/carbon_price/4_tif/non_ag.tif failed: Permission denied

In [ ]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high\2050\xr_cost_transition_ag2ag_diff_carbon_high_2050.nc"
input_xr = xr.open_dataarray(input_path)
input_arr = input_xr.sum(dim=[d for d in input_xr.dims if d != "cell"]).to_numpy()
output_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\transition_ag2ag.tif"
npy_to_map(input_arr, output_tif, proj_file)

In [ ]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high\2050\xr_transition_cost_ag2non_ag_amortised_diff_carbon_high_2050.nc"
input_xr = xr.open_dataarray(input_path)
input_arr = input_xr.sum(dim=[d for d in input_xr.dims if d != "cell"]).to_numpy()
output_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\transition_ag2non_ag.tif"
npy_to_map(input_arr, output_tif, proj_file)

In [31]:
input_path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\0_base_data\carbon_high\2050\xr_carbon_high_price_2050.nc"
input_xr = xr.open_dataarray(input_path)
input_arr = input_xr.sum(dim=[d for d in input_xr.dims if d != "cell"]).to_numpy()
output_tif = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250908_Paper2_Results_NCI\carbon_price\4_tif\price.tif"
npy_to_map(input_arr, output_tif, proj_file)

'N:\\LUF-Modelling\\LUTO2_XH\\LUTO2\\output\\20250908_Paper2_Results_NCI\\carbon_price\\4_tif\\price.tif'